In [1]:
import sys
sys.version

'3.11.10 | packaged by conda-forge | (main, Oct 16 2024, 01:26:25) [Clang 17.0.6 ]'

In [2]:
# Import the SparkSession module
from pyspark.sql import SparkSession
import streamlit as st
import pandas as pd
import os
import io
import urllib3
from minio import Minio

In [4]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# The local installation of Hadoop is version 3.2.0, therefore the environment variable PYSPARK_SUBMIT_ARGS is set to 
# use that version.
os.environ['PYSPARK_SUBMIT_ARGS'] = ' — packages "org.apache.hadoop:hadoop-aws:3.2.1" pyspark-shell'

In [5]:
conf = conf \
    .set('spark.driver.extraJavaOptions', '-Dcom.amazonaws.services.s3.enableV4=true') \
    .set('spark.executor.extraJavaOptions', '-Dcom.amazonaws.services.s3.enableV4=true')
    
sc = SparkContext(conf=conf)

NameError: name 'conf' is not defined

In [6]:
conf = SparkConf()\
    .setAppName('read_data_from_s3_example') \
    .setMaster('local[*]') \
    .set('spark.hadoop.fs.s3a.endpoint', 'http://127.0.0.1:9000') \
    .set('spark.hadoop.fs.s3a.access.key', 'health') \
    .set('spark.hadoop.fs.s3a.secret.key', 'NOentry#23') \
    .set('spark.hadoop.fs.s3a.path.style.access', 'true') \
    .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false') \
    .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .set('spark.driver.extraJavaOptions', '-Dcom.amazonaws.services.s3.enableV4=true -Dlog4j.configuration=file:/Users/ben/.log4j.properties') \
    .set('spark.executor.extraJavaOptions', '-Dcom.amazonaws.services.s3.enableV4=true -Dlog4j.configuration=file:/Users/ben/.log4j.properties')
sc = SparkContext(conf=conf)

Exception in thread "main" org.apache.spark.SparkException: Failed to get main class in JAR with error 'File file:/Users/ben/Library/CloudStorage/OneDrive-UniversityofEdinburgh/Documents/Courses/Big data analytics/Projects/Big Data/— does not exist'.  Please specify one with --class.
	at org.apache.spark.deploy.SparkSubmit.error(SparkSubmit.scala:972)
	at org.apache.spark.deploy.SparkSubmit.prepareSubmitEnvironment(SparkSubmit.scala:486)
	at org.apache.spark.deploy.SparkSubmit.org$apache$spark$deploy$SparkSubmit$$runMain(SparkSubmit.scala:898)
	at org.apache.spark.deploy.SparkSubmit.doRunMain$1(SparkSubmit.scala:180)
	at org.apache.spark.deploy.SparkSubmit.submit(SparkSubmit.scala:203)
	at org.apache.spark.deploy.SparkSubmit.doSubmit(SparkSubmit.scala:90)
	at org.apache.spark.deploy.SparkSubmit$$anon$2.doSubmit(SparkSubmit.scala:1043)
	at org.apache.spark.deploy.SparkSubmit$.main(SparkSubmit.scala:1052)
	at org.apache.spark.deploy.SparkSubmit.main(SparkSubmit.scala)


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [12]:
#aws_access_key = os.environ.get('AWS_SECRET_ID')
#aws_secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', 'health')
hadoopConf.set('fs.s3a.secret.key', 'NOentry#23')
hadoopConf.set('fs.s3a.endpoint', '127.0.0.1')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

NameError: name 'sc' is not defined

In [ ]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkApp").master("local").config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider").getOrCreate()

In [6]:
# Disable SSL verification warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Setting the environment variables
os.environ["AWS_ACCESS_KEY_ID"] = "health"
os.environ["AWS_SECRET_ACCESS_KEY"] = "NOentry#23"
os.environ["AWS_SSL_ENABLED"] = "TRUE"
os.environ["AWS_S3_ENDPOINT"] = "127.0.0.1:9000"

# Initialize Minio client
client = Minio(
    endpoint=os.environ["AWS_S3_ENDPOINT"],
    access_key=os.environ["AWS_ACCESS_KEY_ID"],
    secret_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    secure=True,
    http_client=urllib3.PoolManager(cert_reqs='CERT_NONE')
)

In [7]:
# Get files from the MinIO server
bucket_name = 'templategenerator'

# Read the CSV file into a DataFrame
iris_csv = client.get_object('templategenerator', "iris.csv")
st.success("Data loaded successfully")

# Load the iris dataset into a Pandas DataFrame
iris_df = pd.read_csv(io.BytesIO(iris_csv.read()))

# Convert the Pandas DataFrame to a Spark DataFrame
iris_spark_df = spark.createDataFrame(iris_df)
    
# Show data
iris_spark_df.show()

2024-11-03 18:09:32.051 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 18:09:32.135 
  command:

    streamlit run /Users/ben/miniforge3/envs/big-data/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-11-03 18:09:32.136 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
|         5.4|        3.9|         1.7|        0.4| setosa|
|         4.6|        3.4|         1.4|        0.3| setosa|
|         5.0|        3.4|         1.5|        0.2| setosa|
|         4.4|        2.9|         1.4|        0.2| setosa|
|         4.9|        3.1|         1.5|        0.1| setosa|
|         5.4|        3.7|         1.5|        0.2| setosa|
|         4.8|        3.4|         1.6|        0.2| setosa|
|         4.8|        3.0|         1.4|        0.1| setosa|
|         4.3|        3.0|         1.1| 